In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import mplfinance as mpf
import time

timeS=time.time()


page=1
code=input('종목코드입력:')
url='https://finance.naver.com/item/sise_day.nhn?code='+ code +'&page='

driver=webdriver.Chrome('../chromedriver')
driver.get(url+str(page))
html = bs(driver.page_source, 'html.parser')
endPage=html.find('td', class_="pgRR").find('a')['href'].split('=')[-1]

nav_finance=[]

for page in range(1, int(endPage)+1):
    driver.get(url+str(page))
    html = bs(driver.page_source, 'html.parser')
    soup_tr=html.find_all('tr')
    for i in range(2, 15):
        day_data=[]
        soup_td=soup_tr[i].find_all('td')
        if len(soup_td) < 7 :
            continue
        else:
            for td_txt in soup_td:
                if td_txt.text == u'\xa0':
                    continue
                else:
                    day_data.append(td_txt.text.replace('.', '-').replace('\n','').replace('\t', '').replace(',', ''))
                    # day_data.append(re('[가-힣0-9a-zA-Z]+',td_txt.text))

        nav_finance.append(day_data)

driver.close()

df=pd.DataFrame(nav_finance, columns=['날짜','종가',"전일비",'시가','고가','저가','거래량'])
print(df.info())

df=df.dropna()  # None(NaN) 값 제거

df['날짜']=pd.to_datetime(df['날짜'])
df=df.astype({'종가':'int',"전일비":'int','시가':'int','고가':'int','저가':'int','거래량':'int'})

print(df.info())
print(df.tail())

df.to_csv('./셀트리온.csv', index=False)
df=df.iloc[:30]

df = df.rename(columns={'날짜':'Date', '시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})
df = df.sort_values(by='Date')
df.index = pd.to_datetime(df.Date)
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

#mpf.plot(df, title='Celltrion candle chart', type='candle')
#mpf.plot(df, title='Celltrion ohlc chart', type='ohlc')

kwargs = dict(title='Celltrion customized chart', type='candle',
              mav=(2, 4, 6), volume=True, ylabel='ohlc candles')

mc = mpf.make_marketcolors(up='r', down='b', inherit=True)
s  = mpf.make_mpf_style(marketcolors=mc)
mpf.plot(df, **kwargs, style=s)

종목코드입력:068270


TypeError: object of type 'NoneType' has no len()

In [ ]:
# !pip install mplfinance